In [0]:
# %pip install dbldatagen

# import dbldatagen as dg
# from pyspark.sql import functions as F, types as T
# from pyspark.sql.window import Window

# def drop_volume(chapter_number: str):
#     spark.sql(f"drop volume if exists workspace.default.chapter_{chapter_number}")

# def create_volume(chapter_number: str):
#     spark.sql(f"""
#         create volume if not exists workspace.default.chapter_{chapter_number}
#         comment 'Datasets for chapter {chapter_number}'
#     """)

# def create_enumerated_files(temp_path: str, table_name: str, file_format: str):
#     data_files = [f for f in dbutils.fs.ls(temp_path) if f.name.startswith("part-")]
#     for index, file_name in enumerate(data_files, start=1):
#         path = temp_path.replace('temp/', '')
#         dbutils.fs.mv(file_name.path, f"{path}{table_name}_0{index}.{file_format}")

# def create_complex_data_files(chapter_number: str, temp_root: str):
#     od_path = f"/Volumes/workspace/default/chapter_{chapter_number}/order_details/parquet/"
#     df = spark.read.parquet(od_path)

#     array_products = (
#         df.groupBy("order_id")
#           .agg(F.collect_list("product_id").alias("array_products"))
#     )

#     order_details_dict = (
#         df.groupBy("order_id")
#           .agg(F.collect_list(F.struct(
#                 F.col("product_id").alias("productid"),
#                 F.col("unit_price").alias("unitprice"),
#                 F.col("discount").alias("discount")
#           )).alias("order_details"))
#     )

#     (array_products.repartition(4)
#         .write.mode("overwrite").format("json")
#         .save(f"{temp_root}/array_products_by_order/json/"))

#     (order_details_dict.repartition(4)
#         .write.mode("overwrite").format("json")
#         .save(f"{temp_root}/order_details_dict/json/"))

# def write_all_formats(df, main_temp_path: str, table_name: str, one_file: bool = False):
#     dict_types = { 
#         "csv":     {"options": {"delimiter": ",", "header": "true"}},
#         "parquet": {"options": {}},
#         "json":    {"options": {}},
#         "avro":    {"options": {}},
#         "orc":     {"options": {}}
#     }
#     for fmt, spec in dict_types.items():
#         tmp = f"{main_temp_path}/{table_name}/{fmt}/"
#         writer_df = df.coalesce(1) if one_file else df.repartition(4)
#         (writer_df.write
#             .mode("overwrite")
#             .options(**spec["options"])
#             .format(fmt)
#             .save(tmp))
#         create_enumerated_files(tmp, table_name, fmt)

# def generate_and_write_to_volume(
#     chapter_number: str,
#     users_n: int = 1000,
#     orders_n: int = 500,
#     orphan_rate: float = 0.01,
#     seed_users: int = 101,
#     seed_orders: int = 202,
#     seed_details: int = 303,
#     seed_lines: int = 404,
#     seed_pickers: int = 505,
#     ):

#     countries = [
#         ("US","United States"), ("BR","Brazil"), ("IN","India"), ("GB","United Kingdom"),
#         ("DE","Germany"), ("FR","France"), ("ES","Spain"), ("CA","Canada"),
#         ("AU","Australia"), ("MX","Mexico"), ("AR","Argentina"), ("CO","Colombia"),
#         ("PT","Portugal"), ("IT","Italy"), ("NL","Netherlands"), ("SE","Sweden"),
#         ("JP","Japan"), ("KR","South Korea"), ("ZA","South Africa"), ("NG","Nigeria")
#     ]
#     country_codes  = [c for c,_ in countries]
#     country_weights = [10, 18, 18, 5, 4, 4, 4, 3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2]

#     user_professions = ['Data Engineer','Data Architect','Data Analyst','Data Scientist','Developer']

#     courses = [
#         'Building a Data Lakehouse with SQL and DDP',
#         'Mastering SQL on Databricks',
#         'Data Heroes Mentorship Program',
#         'Book Club - Spark the Definitive Guide',
#         'Book Club - Delta Lake the Definitive Guide'
#     ]
#     course_distribution = [15, 21, 30, 19, 15]
#     course_prices = [99.00, 297.00, 997.00, 297.00, 297.00]

#     email_providers = ['@gmail.com','@hotmail.com','@outlook.com']
#     email_distribution = [60, 25, 15]

#     gender = ['M','F']
#     gender_distribution = [49, 51]

#     payment_methods = ['credit_card','debit_card','paypal','apple_pay','google_pay']
#     payment_method_distribution = [55, 15, 12, 8, 10]

#     discounts = [0.05, 0.10, 0.15]
#     discount_distribution = [50, 35, 15]

#     installments = [1, 2, 3, 10, 12]
#     installment_distribution = [20, 15, 5, 30, 30]

#     invalid_count = int(orders_n * orphan_rate)
#     invalid_user_id = users_n + 9999

#     drop_volume(chapter_number)
#     create_volume(chapter_number)
#     main_temp_path = f"/Volumes/workspace/default/chapter_{chapter_number}/temp"

#     gen_users = (
#         dg.DataGenerator(spark, name="users_gen", rows=users_n, partitions=4,
#                          randomSeedMethod="hash_fieldname")
#         .withIdOutput()
#         .withColumn("user_id", T.LongType(), expr="id + 1")
#         .withColumn("email_provider", T.StringType(), values=email_providers,
#                     weights=email_distribution, baseColumn=["user_id"])
#         .withColumn("email", T.StringType(),
#                     expr="concat('user_', lpad(cast(user_id as string), 6, '0'), email_provider)",
#                     baseColumn=["user_id","email_provider"])
#         .withColumn("gender", T.StringType(), values=gender,
#                     weights=gender_distribution, baseColumn=["user_id"],
#                     nullable=True, percentNulls=0.08)
#         .withColumn("profession", T.StringType(), values=user_professions, baseColumn=["user_id"],
#                     nullable=True, percentNulls=0.05)
#         .withColumn("country_code", T.StringType(), values=country_codes,
#                     weights=country_weights, baseColumn=["user_id"])
#     )
#     users_df = gen_users.build().select("user_id","email","gender","profession","country_code")
#     country_map = F.create_map([F.lit(x) for kv in countries for x in kv])
#     users_df = users_df.withColumn("country", country_map[F.col("country_code")]) \
#                        .select("user_id","email","gender","profession","country_code","country")

#     products_df = spark.createDataFrame(
#         list(zip(range(1, len(courses)+1), courses, course_prices)),
#         schema="product_id long, product_name string, base_price double"
#     )

#     day_offset_expr = f"cast(pmod(abs(xxhash64(order_id, {seed_orders})), 365) as int)"
#     sec_of_day_expr = f"(cast(pmod(abs(xxhash64(order_id, {seed_orders}+1)), 86399) as int) + 1)"
#     order_ts_expr = (
#         f"to_timestamp(from_unixtime(unix_timestamp('2025-01-01 00:00:00')"
#         f" + {day_offset_expr} * 86400 + {sec_of_day_expr}))"
#     )

#     gen_orders = (
#         dg.DataGenerator(spark, name="orders_gen", rows=orders_n, partitions=4,
#                          randomSeedMethod="hash_fieldname")
#         .withIdOutput()
#         .withColumn("order_id", T.LongType(), expr="id + 1")
#         .withColumn("user_id", T.LongType(), minValue=1, maxValue=users_n)
#         .withColumn("order_date", T.TimestampType(), expr=order_ts_expr)
#         .withColumn("payment_method", T.StringType(), values=payment_methods,
#                     weights=payment_method_distribution)
#         .withColumn("installments", T.IntegerType(), values=installments,
#                     weights=installment_distribution)
#     )

#     orders_valid = gen_orders.build().select(
#         "order_id","user_id","order_date","payment_method","installments"
#     )

#     orphans_ids = (
#         orders_valid
#         .select("order_id", F.xxhash64("order_id", F.lit(seed_orders)).alias("h"))
#         .orderBy("h").limit(invalid_count)
#         .select("order_id").withColumn("is_orphan", F.lit(True))
#     )
#     orders_df = (
#         orders_valid.join(orphans_ids, "order_id", "left")
#                     .withColumn("user_id",
#                         F.when(F.col("is_orphan"), F.lit(invalid_user_id)).otherwise(F.col("user_id")).cast("long"))
#                     .drop("is_orphan")
#     )

#     weights_lines = [20, 25, 30, 15, 10]
#     cum_lines = [sum(weights_lines[:i+1]) for i in range(len(weights_lines))]
#     bucket_lines = F.pmod(F.xxhash64(F.col("order_id"), F.lit(seed_lines)), F.lit(100)).cast("int")

#     order_with_count = (
#         orders_df.select("order_id","user_id")
#         .withColumn(
#             "line_count",
#             F.when(bucket_lines < cum_lines[0], F.lit(1))
#              .when(bucket_lines < cum_lines[1], F.lit(2))
#              .when(bucket_lines < cum_lines[2], F.lit(3))
#              .when(bucket_lines < cum_lines[3], F.lit(4))
#              .otherwise(F.lit(5))
#         )
#     )
#     order_lines = (
#         order_with_count
#         .withColumn("line_no", F.explode(F.sequence(F.lit(1), F.col("line_count"))))
#         .drop("line_count")
#     )

#     w_user = Window.partitionBy("user_id").orderBy(F.col("order_id").asc(), F.col("line_no").asc())
#     order_lines_ranked = order_lines.withColumn("user_line_rank", F.row_number().over(w_user)) \
#                                     .filter(F.col("user_line_rank") <= 5)

#     h = [F.xxhash64(F.col("user_id"), F.lit(i), F.lit(seed_pickers)) for i in range(1, 6)]
#     perm_array = F.array_sort(F.array(
#         F.struct(h[0].alias("h"), F.lit(1).alias("pid")),
#         F.struct(h[1].alias("h"), F.lit(2).alias("pid")),
#         F.struct(h[2].alias("h"), F.lit(3).alias("pid")),
#         F.struct(h[3].alias("h"), F.lit(4).alias("pid")),
#         F.struct(h[4].alias("h"), F.lit(5).alias("pid"))
#     ))
#     order_lines_with_product = (
#         order_lines_ranked
#         .withColumn("product_struct", F.element_at(perm_array, F.col("user_line_rank")))
#         .withColumn("product_id", F.col("product_struct.pid").cast("long"))
#         .drop("product_struct")
#     )

#     cum_disc = [sum(discount_distribution[:i+1]) for i in range(len(discount_distribution))]
#     bucket_disc = F.pmod(F.xxhash64(F.col("order_id"), F.col("line_no"), F.lit(seed_details+1)), F.lit(100)).cast("int")
#     discount_col = (
#         F.when(bucket_disc < cum_disc[0], F.lit(discounts[0]))
#          .when(bucket_disc < cum_disc[1], F.lit(discounts[1]))
#          .otherwise(F.lit(discounts[2]))
#     )
#     details_df = (
#         order_lines_with_product
#         .withColumn("discount", discount_col.cast("double"))
#         .join(products_df.select("product_id","base_price"), "product_id", "inner")
#         .withColumn("unit_price", F.col("base_price"))
#         .select("order_id","product_id","unit_price","discount")
#     )

#     write_all_formats(users_df,    main_temp_path, "users",         one_file=False)
#     write_all_formats(products_df, main_temp_path, "products",      one_file=True)
#     write_all_formats(orders_df,   main_temp_path, "orders",        one_file=False)
#     write_all_formats(details_df,  main_temp_path, "order_details", one_file=False)

#     create_complex_data_files(chapter_number, main_temp_path)
#     create_enumerated_files(f"{main_temp_path}/array_products_by_order/json/", "array_products_by_order", "json")
#     create_enumerated_files(f"{main_temp_path}/order_details_dict/json/",       "order_details_dict",       "json")

#     dbutils.fs.rm(main_temp_path, True)

In [0]:
%pip install dbldatagen

import dbldatagen as dg
from pyspark.sql import functions as F, types as T
from pyspark.sql.window import Window

def drop_volume(chapter_number: str):
    spark.sql(f"drop volume if exists workspace.default.chapter_{chapter_number}")

def create_volume(chapter_number: str):
    spark.sql(f"""
        create volume if not exists workspace.default.chapter_{chapter_number}
        comment 'Datasets for chapter {chapter_number}'
    """)

def create_enumerated_files(temp_path: str, table_name: str, file_format: str):
    data_files = [f for f in dbutils.fs.ls(temp_path) if f.name.startswith("part-")]
    for index, file_name in enumerate(data_files, start=1):
        path = temp_path.replace('temp/', '')
        dbutils.fs.mv(file_name.path, f"{path}{table_name}_0{index}.{file_format}")

def create_complex_data_files(chapter_number: str, temp_root: str):
    od_path = f"/Volumes/workspace/default/chapter_{chapter_number}/order_details/parquet/"
    df = spark.read.parquet(od_path)

    array_products = (
        df.groupBy("order_id")
          .agg(F.collect_list("product_id").alias("array_products"))
    )

    order_details_dict = (
        df.groupBy("order_id")
          .agg(F.collect_list(F.struct(
                F.col("product_id").alias("productid"),
                F.col("unit_price").alias("unitprice"),
                F.col("discount").alias("discount")
          )).alias("order_details"))
    )

    (array_products.repartition(4)
        .write.mode("overwrite").format("json")
        .save(f"{temp_root}/array_products_by_order/json/"))

    (order_details_dict.repartition(4)
        .write.mode("overwrite").format("json")
        .save(f"{temp_root}/order_details_dict/json/"))

def write_all_formats(df, main_temp_path: str, table_name: str, one_file: bool = False):
    dict_types = { 
        "csv":     {"options": {"delimiter": ",", "header": "true"}},
        "parquet": {"options": {}},
        "json":    {"options": {}},
        "avro":    {"options": {}},
        "orc":     {"options": {}}
    }
    for fmt, spec in dict_types.items():
        tmp = f"{main_temp_path}/{table_name}/{fmt}/"
        writer_df = df.coalesce(1) if one_file else df.repartition(4)
        (writer_df.write
            .mode("overwrite")
            .options(**spec["options"])
            .format(fmt)
            .save(tmp))
        create_enumerated_files(tmp, table_name, fmt)

def generate_and_write_to_volume(
    chapter_number: str,
    users_n: int = 1000,
    orders_n: int = 500,
    orphan_rate: float = 0.01,
    seed_users: int = 101,
    seed_orders: int = 202,
    seed_details: int = 303,
    seed_lines: int = 404,
    seed_pickers: int = 505,
):

    countries = [
        ("US","United States"), ("BR","Brazil"), ("IN","India"), ("GB","United Kingdom"),
        ("DE","Germany"), ("FR","France"), ("ES","Spain"), ("CA","Canada"),
        ("AU","Australia"), ("MX","Mexico"), ("AR","Argentina"), ("CO","Colombia"),
        ("PT","Portugal"), ("IT","Italy"), ("NL","Netherlands"), ("SE","Sweden"),
        ("JP","Japan"), ("KR","South Korea"), ("ZA","South Africa"), ("NG","Nigeria")
    ]
    country_codes  = [c for c,_ in countries]
    country_weights = [10, 18, 18, 5, 4, 4, 4, 3, 2, 3, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2]

    user_professions = ['Data Engineer','Data Architect','Data Analyst','Data Scientist','Developer']
    courses = [
        'Building a Data Lakehouse with SQL and DDP',
        'Mastering SQL on Databricks',
        'Data Heroes Mentorship Program',
        'Book Club - Spark the Definitive Guide',
        'Book Club - Delta Lake the Definitive Guide'
    ]
    course_prices = [99.00, 297.00, 997.00, 297.00, 297.00]

    email_providers = ['@gmail.com','@hotmail.com','@outlook.com']
    email_distribution = [60, 25, 15]
    gender = ['M','F']
    gender_distribution = [49, 51]
    payment_methods = ['credit_card','debit_card','paypal','apple_pay','google_pay']
    payment_method_distribution = [55, 15, 12, 8, 10]
    discounts = [0.05, 0.10, 0.15]
    discount_distribution = [50, 35, 15]
    installments = [1, 2, 3, 10, 12]
    installment_distribution = [20, 15, 5, 30, 30]

    invalid_count = int(orders_n * orphan_rate)
    invalid_user_id = users_n + 9999

    drop_volume(chapter_number)
    create_volume(chapter_number)
    main_temp_path = f"/Volumes/workspace/default/chapter_{chapter_number}/temp"

    # USERS — garantir ordem com baseColumn
    gen_users = (
        dg.DataGenerator(spark, name="users_gen", rows=users_n, partitions=4, randomSeedMethod="hash_fieldname")
        .withIdOutput()
        .withColumn("user_id", T.LongType(), expr="id + 1")
        .withColumn("email_provider", T.StringType(), values=email_providers, weights=email_distribution, baseColumn=["user_id"])
        .withColumn(
            "email", T.StringType(),
            expr="concat('user_', lpad(cast(user_id as string), 6, '0'), email_provider)",
            baseColumn=["user_id", "email_provider"]
        )
        .withColumn("gender", T.StringType(), values=gender, weights=gender_distribution, baseColumn=["user_id"],
                    nullable=True, percentNulls=0.08)
        .withColumn("profession", T.StringType(), values=user_professions, baseColumn=["user_id"],
                    nullable=True, percentNulls=0.05)
        .withColumn("country_code", T.StringType(), values=country_codes, weights=country_weights, baseColumn=["user_id"])
    )
    users_df = gen_users.build().select("user_id","email","gender","profession","country_code")
    country_map = F.create_map([F.lit(x) for kv in countries for x in kv])
    users_df = users_df.withColumn("country", country_map[F.col("country_code")]) \
                       .select("user_id","email","gender","profession","country_code","country")

    # PRODUCTS
    products_df = spark.createDataFrame(
        list(zip(range(1, len(courses)+1), courses, course_prices)),
        schema="product_id long, product_name string, base_price double"
    )

    # ORDERS — order_id e user_id com seeds diferentes + timestamp não-meia-noite
    day_offset_expr = f"cast(pmod(abs(xxhash64(id + 1, {seed_orders})), 365) as int)"
    sec_of_day_expr = f"(cast(pmod(abs(xxhash64(id + 1, {seed_orders}+1)), 86399) as int) + 1)"
    order_ts_expr = (
        f"to_timestamp(from_unixtime(unix_timestamp('2025-01-01 00:00:00')"
        f" + {day_offset_expr} * 86400 + {sec_of_day_expr}))"
    )

    gen_orders = (
        dg.DataGenerator(spark, name="orders_gen", rows=orders_n, partitions=4, randomSeedMethod="hash_fieldname")
        .withIdOutput()
        .withColumn("order_id", T.LongType(), expr="id + 1")
        .withColumn("user_id", T.LongType(), expr=f"pmod(abs(xxhash64(id + 1, {seed_users})), {users_n}) + 1")
        .withColumn("order_date", T.TimestampType(), expr=order_ts_expr)
        .withColumn("payment_method", T.StringType(), values=payment_methods, weights=payment_method_distribution)
        # valor preliminar de installments (será substituído por regra realista)
        .withColumn("installments", T.IntegerType(), values=installments, weights=installment_distribution)
    )

    # Build orders e aplicar regra realista de parcelamento com base em payment_method + país do usuário
    orders_valid = gen_orders.build().select("order_id","user_id","order_date","payment_method","installments")

    # Trazer country_code do usuário para aplicar regra
    orders_with_country = (
        orders_valid.join(users_df.select("user_id","country_code"), "user_id", "left")
    )

    # Hashes determinísticos para decidir se parcela e quantas parcelas (2–12)
    decide_parcelar = F.pmod(F.abs(F.xxhash64(F.col("order_id"), F.lit(seed_orders + 7))), F.lit(100)).cast("int")
    parcelas_hash   = F.pmod(F.abs(F.xxhash64(F.col("order_id"), F.lit(seed_orders + 8))), F.lit(11)).cast("int")  # 0..10

    # países que suportam parcelamento típico via cartão
    latam_countries = F.array(F.lit("BR"), F.lit("MX"), F.lit("AR"))

    installments_realistic = (
        F.when(
            (F.col("payment_method") == "credit_card") &
            (F.array_contains(latam_countries, F.col("country_code"))),
            F.when(decide_parcelar < 30, F.lit(1))  # ~30% 1x
             .otherwise(parcelas_hash + 2)          # 2..12
        )
        .otherwise(F.lit(1))  # PayPal, débito, Apple Pay, Google Pay e demais países: 1x
    )

    orders_df = (
        orders_with_country
        .withColumn("installments", installments_realistic.cast("int"))
    )

    # ORPHANS (injeta user_id inválido em fração dos pedidos)
    orphans_ids = (
        orders_df.select("order_id", F.xxhash64("order_id", F.lit(seed_orders)).alias("h"))
                 .orderBy("h").limit(invalid_count)
                 .select("order_id").withColumn("is_orphan", F.lit(True))
    )

    orders_df = (
        orders_df.join(orphans_ids, "order_id", "left")
                 .withColumn("user_id",
                     F.when(F.col("is_orphan"), F.lit(invalid_user_id)).otherwise(F.col("user_id")).cast("long"))
                 .drop("is_orphan", "country_code")  # country_code não precisa ir para a tabela final
    )

    # ORDER DETAILS
    weights_lines = [20, 25, 30, 15, 10]
    cum_lines = [sum(weights_lines[:i+1]) for i in range(len(weights_lines))]
    bucket_lines = F.pmod(F.xxhash64(F.col("order_id"), F.lit(seed_lines)), F.lit(100)).cast("int")

    order_with_count = (
        orders_df.select("order_id","user_id")
        .withColumn("line_count",
            F.when(bucket_lines < cum_lines[0], F.lit(1))
             .when(bucket_lines < cum_lines[1], F.lit(2))
             .when(bucket_lines < cum_lines[2], F.lit(3))
             .when(bucket_lines < cum_lines[3], F.lit(4))
             .otherwise(F.lit(5)))
    )
    order_lines = order_with_count.withColumn("line_no", F.explode(F.sequence(F.lit(1), F.col("line_count")))).drop("line_count")

    w_user = Window.partitionBy("user_id").orderBy(F.col("order_id").asc(), F.col("line_no").asc())
    order_lines_ranked = order_lines.withColumn("user_line_rank", F.row_number().over(w_user)).filter(F.col("user_line_rank") <= 5)

    h = [F.xxhash64(F.col("user_id"), F.lit(i), F.lit(seed_pickers)) for i in range(1, 6)]
    perm_array = F.array_sort(F.array(
        F.struct(h[0].alias("h"), F.lit(1).alias("pid")),
        F.struct(h[1].alias("h"), F.lit(2).alias("pid")),
        F.struct(h[2].alias("h"), F.lit(3).alias("pid")),
        F.struct(h[3].alias("h"), F.lit(4).alias("pid")),
        F.struct(h[4].alias("h"), F.lit(5).alias("pid"))
    ))

    order_lines_with_product = (
        order_lines_ranked
        .withColumn("product_struct", F.element_at(perm_array, F.col("user_line_rank")))
        .withColumn("product_id", F.col("product_struct.pid").cast("long"))
        .drop("product_struct")
    )

    cum_disc = [sum(discount_distribution[:i+1]) for i in range(len(discount_distribution))]
    bucket_disc = F.pmod(F.xxhash64(F.col("order_id"), F.col("line_no"), F.lit(seed_details+1)), F.lit(100)).cast("int")
    discount_col = (
        F.when(bucket_disc < cum_disc[0], F.lit(discounts[0]))
         .when(bucket_disc < cum_disc[1], F.lit(discounts[1]))
         .otherwise(F.lit(discounts[2]))
    )

    details_df = (
        order_lines_with_product
        .withColumn("discount", discount_col.cast("double"))
        .join(products_df.select("product_id","base_price"), "product_id", "inner")
        .join(orders_df.select("order_id","installments"), "order_id", "inner")
        .withColumn("unit_price", F.round((F.col("base_price") * (1 - F.col("discount"))) / F.col("installments"), 2))
        .select("order_id","product_id","unit_price","discount")
    )

    # WRITE
    write_all_formats(users_df, main_temp_path, "users")
    write_all_formats(products_df, main_temp_path, "products", one_file=True)
    write_all_formats(orders_df.select("order_id","user_id","order_date","payment_method","installments"),
                      main_temp_path, "orders")
    write_all_formats(details_df,  main_temp_path, "order_details")

    # DERIVADOS COMPLEXOS
    create_complex_data_files(chapter_number, main_temp_path)
    create_enumerated_files(f"{main_temp_path}/array_products_by_order/json/", "array_products_by_order", "json")
    create_enumerated_files(f"{main_temp_path}/order_details_dict/json/", "order_details_dict", "json")

    # CLEANUP TEMP
    dbutils.fs.rm(main_temp_path, True)


In [0]:
generate_and_write_to_volume(chapter_number="05")

In [0]:
# display(spark.read.parquet('/Volumes/workspace/default/chapter_05/orders/parquet'))
# display(spark.read.parquet('/Volumes/workspace/default/chapter_05/users/parquet'))
# display(spark.read.parquet('/Volumes/workspace/default/chapter_05/order_details/parquet'))
# display(spark.read.parquet('/Volumes/workspace/default/chapter_05/products/parquet'))

In [0]:
spark.read.parquet('/Volumes/workspace/default/chapter_05/orders/parquet').createOrReplaceTempView('orders')
spark.read.parquet('/Volumes/workspace/default/chapter_05/order_details/parquet').createOrReplaceTempView('order_details')
spark.read.parquet('/Volumes/workspace/default/chapter_05/users/parquet').createOrReplaceTempView('users')
spark.read.parquet('/Volumes/workspace/default/chapter_05/products/parquet').createOrReplaceTempView('products')

In [0]:
%sql 

SELECT od.*
FROM orders as o 
join order_details as od on o.order_id = od.order_id
join users as u on o.user_id = u.user_id
join products as p on p.product_id = od.product_id
order by o.order_date